In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [36]:
df = pd.read_csv('BTCUSDT-trades-2023-09.csv')
df=df.sort_values(by='time')
df

MemoryError: Unable to allocate 1.14 GiB for an array with shape (2, 76683300) and data type int64

In [ ]:
def convert_tick_to_ohlcv(data):
    """
    Converts given Binance tick data into 1-hour interval OHLCV (Open, High, Low, Close, Volume) data.
    :param data: DataFrame with Tick data
    :return: DataFrame with the Open, High, Low, Close, Volume values
    """
    data['time'] = pd.to_datetime(data['time'], unit='ms')

    ohlcv = data.resample('1min', on='time').agg({
        'price': ['first', 'max', 'min', 'last'],
        'qty': 'sum'


    })
    ohlcv.columns = ['Open', 'High', 'Low', 'Close', 'Volume']


    return ohlcv

In [ ]:
ohlcv=convert_tick_to_ohlcv(df)
ohlcv

## rsi 구하는 함수

In [ ]:
def calculate_rsi(prices, period=14):
    # 주식 가격의 차이 계산
    delta = pd.Series(prices).diff(1).dropna()

    # 상승한 가격의 차이만 선택
    gain = delta.where(delta > 0, 0)

    # 하락한 가격의 차이만 선택
    loss = -delta.where(delta < 0, 0)

    # 이동평균 계산
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    # 상대적인 강도 계산
    rs = avg_gain / avg_loss

    # RSI 계산
    rsi = 100 - (100 / (1 + rs))

    return rsi

# 가상의 주식 가격 데이터 생성
prices = [100, 105, 110, 95, 102, 98, 105]

# RSI 계산
rsi_values = calculate_rsi(prices)

# 결과 출력
print(rsi_values)


In [ ]:
def calculate_volatility(data, window=20):
    """
    Calculate the rolling volatility using the standard deviation of returns.
    :param data: DataFrame with OHLCV data
    :param window: The number of periods to use for calculating the standard deviation
    :return: DataFrame with the volatility values
    """

    # Calculate daily returns
    data['returns'] = data['Close'].pct_change()

    # Calculate the rolling standard deviation of returns
    data['volatility'] = data['returns'].rolling(window=window).std()
    # 이동평균 (Moving Average) 추가
    data['MA'] = data['Close'].rolling(window=3, min_periods=1).mean()

    # RSI (Relative Strength Index) 추가
    data['RSI'] = calculate_rsi(data['Close'], period=14)

    return data

In [ ]:
ohlcv

In [ ]:
result=calculate_volatility(ohlcv)
result

In [ ]:
result.to_csv('rsi9.csv',index=True)

In [ ]:
rsi9=pd.read_csv('rsi9.csv')